In [3]:
import pandas as pd
import sklearn
PseudoData = pd.read_csv('PseudoData.csv')  #Read in CSV

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
PseudoData

,Student,Grade,GPA,#Honors semesters,#AP semesters,Days Enrolled,Days Present,Att Rate
0,20000000,12,4.000,0,0,169,149.0,88.2
1,20000001,12,3.400,0,0,169,150.0,88.8
2,20000002,12,4.000,0,1,169,128.0,75.7
3,20000003,12,3.750,1,3,169,151.0,89.3
4,20000004,12,3.500,0,4,169,165.5,97.9
5,20000005,11,3.571,0,0,169,160.0,94.7
6,20000006,11,3.231,0,10,169,168.0,99.4
7,20000007,11,4.000,0,0,169,154.0,91.1
8,20000008,11,3.750,0,0,169,162.0,95.9
9,20000009,10,4.000,2,6,169,143.0,84.6
